# H₂ Molecule Homework Assignment
### Quantum Software Development Journey: From Theory to Application with Classiq - Part 3

- Similarly to what we have done in class, in this exercise we will implement the VQE on H2 molecule.
- This time instead of using the built-in methods and functions (such as `Molecule` and `MoleculeProblem`) to difne and solve the problem, you will be provided with a two qubits Hamiltonian.

## Submission
- Submit the completed Jupyter notebook and report via GitHub. Ensure all files are correctly named and organized.
- Use the Typeform link provided in the submission folder to confirm your submission.

## Additional Resources
- [Classiq Documentation](https://docs.classiq.io/latest/)
- The notebook from live session #3

## Important Dates
- **Assignment Release:** 22.5.2024
- **Submission Deadline:** 3.6.2024 (7 A.M GMT+3)

---

Happy coding and good luck!

### Part 1

Given the following Hamiltonian:

$$
\hat{H} = -1.0523 \cdot (I \otimes I) + 0.3979 \cdot (I \otimes Z) - 0.3979 \cdot (Z \otimes I) - 0.0112 \cdot (Z \otimes Z) + 0.1809 \cdot (X \otimes X)
$$

Complete the following code

In [ ]:
!pip install -U classiq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.5/401.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 734.3 kB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successful

In [ ]:
import classiq
classiq.authenticate()

Your user code: LSSS-MJFD
If a browser doesn't automatically open, please visit this URL from any trusted device: https://auth.classiq.io/activate?user_code=LSSS-MJFD


In [16]:
from typing import List
from classiq import *

#TODO: Complete Hamiltonian
HAMILTONIAN = QConstant("HAMILTONIAN", List[PauliTerm], [
    PauliTerm([Pauli.I, Pauli.I], -1.0523),
    PauliTerm([Pauli.I, Pauli.Z], 0.3979),
    PauliTerm([Pauli.Z, Pauli.I], -0.3979),
    PauliTerm([Pauli.Z, Pauli.Z], -0.0112),
    PauliTerm([Pauli.X, Pauli.X], 0.1809),
])

In [17]:
@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    # TODO: Create an ansatz which allows each qubit to have
    # arbitrary rotation
    allocate(2, q)

    U(angles[0], angles[1], angles[2], 0, q[0])
    U(angles[0], angles[1], angles[2], 0, q[1])

@cfunc
def cmain() -> None:
    res = vqe(
        hamiltonian=HAMILTONIAN,
        maximize=False,
        initial_point=[],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

#TODO: complete the line, use classical_execution_function
qmod = create_model(main, classical_execution_function=cmain)

qprog = synthesize(qmod)


In [18]:
execution = execute(qprog)
res = execution.result()
# execution.open_in_ide()
vqe_result = res[0].value


In [19]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.0546669921875
Optimal parameters: {'angles_0': 6.279238566143106, 'angles_1': -0.7105268722399182, 'angles_2': 4.394957787844692}
Eigenstate: {'00': (1+0j)}


Does it similar to the `optimal energy` we calculated in class? \
Does it similar to the `total energy` we calculated in class?

### Part 2

**Now, we want to have a more interesting ansatz in our `main`.**  
Add **one** line of code to the `main` function you created in Part 1 that creates **entanglement** between the two qubits.  
Which gate should you use?

In [20]:
@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    # TODO: Create an ansatz which allows each qubit to have
    # arbitrary rotation
    allocate(2, q)

    U(angles[0], angles[1], angles[2], 0, q[0])
    U(angles[0], angles[1], angles[2], 0, q[1])

    CX(q[0], q[1])


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN, # TODO: complete the missing argument
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

#TODO: complete the line, use classical_execution_function
qmod = create_model(main, classical_execution_function=cmain)

qprog = synthesize(qmod)


In [21]:
execution = execute(qprog)
res = execution.result()
# execution.open_in_ide()
vqe_result = res[0].value


In [22]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.84395341796875
Optimal parameters: {'angles_0': 3.220640081190124, 'angles_1': 0.48857020981030813, 'angles_2': 1.69265245066969}
Eigenstate: {'10': (0.03125+0j), '11': (0.03125+0j), '01': (0.9990229601966113+0j)}


Does it similar to the `optimal energy` we calculated in class? \
Does it similar to the `total energy` we calculated in class? \
What can we learn about the provided  form this result Hamiltonian?